## Importer les données

In [1]:
import pickle
import numpy as np

In [2]:
[data,id2titles,fields]=pickle.load(open("imdb_extrait.pkl","rb"))

### ce que le fichier comprend 
data enregistre 4587 films et leurs discriptions
id2titles est une dictionnaire qui enregistre les id est leurs titles
fields enregistre les catégories du film
### datax prend les premiers 32 critères parmi les 34
### datay enregistre les étiquettes pour celuis dans datax : 1 ou -1

In [3]:
print(data.shape)
print(data[0,:])
print(len(id2titles.keys()))
print(fields.keys())
print(fields.values())

(4587, 34)
[0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 1.0000e+00
 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
 1.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 1.0000e+00
 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 1.0800e+02 2.0000e+00
 1.9880e+03 1.2324e+07 4.2062e+04 5.5000e+00]
4587
dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33])
dict_values(['Sci-Fi', 'Crime', 'Romance', 'Animation', 'Music', 'Comedy', 'War', 'Horror', 'Film-Noir', 'Adult', 'News', 'Reality-TV', 'Thriller', 'Western', 'Mystery', 'Short', 'Talk-Show', 'Drama', 'Action', 'Documentary', 'Musical', 'History', 'Family', 'Adventure', 'Fantasy', 'Game-Show', 'Sport', 'Biography', 'Duree', 'couleur', 'annee', 'budget', 'nb_votes', 'rating'])


In [4]:
datax=data[:,:32]
print(datax.shape)

(4587, 32)


In [5]:
datay=np.array([1 if x[33]>6.5 else -1 for x in data])

In [6]:
datay

array([-1,  1,  1, ...,  1,  1, -1])

## Arbre de décision
une mesure d'homognénéité sur la partition, l'entropie de Shannon ou l'index de Gini
algo de decision:  

    chaque dimension d'espace de description est considérée itérativement  
    pour chaque seuil(split) possible, calculer l'homogénéité des deux partitions obtenues, choisi le meulleur honogeneité  
    

In [7]:
from collections import *

In [8]:
count=Counter(datay)

In [9]:
print(count.keys())

dict_keys([-1, 1])


In [10]:
def entropie(vect):
    H=0
    count=Counter(vect)
    for x in (count.keys()):
        px=count[x]/len(vect)
        H-=(px)*math.log(px)
    return H

In [11]:
def entropie_cond(list_vect):
    #list_vect : une nested listes de labels, c'est la partition
    y = numpy.concatenate([numpy.array(i) for i in list_vect])
    len_all=len(y)
    hcond=0
    for partition in list_vect:
        pp=len(partition)/len_all
        hcond-=pp*entropie(partition)
    return hcond

## Quelques expériences préliminaires

In [38]:
from decisiontree import DecisionTree
#import pydot as py
import math

In [46]:
def randomList(data,p):
    nb=math.floor(data.shape[0]*p)
    train=[]
    
    while(len(train)<nb):
        t=np.random.randint(0,data.shape[0])
        if(t not in train):
            train.append(t)
    return train

In [47]:
x=randomList(datax,0.1)
print(len(x))


458


In [ ]:
def train_test(datax,datay):

In [30]:
dt = DecisionTree ()
dt . max_depth = 5 # on fixe la taille de l ’ arbre a 5
dt . min_samples_split = 2 # nombre minimum d ’ exemples pour spliter un noeud
dt . fit ( datax , datay )

dataxnew=datax [:5 ,:]
dataynew=datay[:5]
#nombre=dt . predict ( datax [:5 ,:])
#print (nombre)
print ( dt . score ( dataxnew , dataynew ))
# dessine l ’ arbre dans un fichier pdf si pydot est installe .
#dt.to_pdf ("/tmp /test_tree . pdf ", fields )
#py.graph_from_dot_data(dt.to_dot(None))[0].write_pdf("/tmp /test_tree . pdf ")
# sinon utiliser http :// www . webgraphviz . com /
dt . to_dot ( fields )
# ou dans la console
print ( dt . print_tree ( fields ))

0.6
var Drama :  > 0.500000 
	var budget :  > 15660000.000000 
		var Duree :  > 115.500000 
			var budget :  > 36500000.000000 
				var Duree :  > 137.500000 
					class : 1 {1: 101, -1: 20}
				var Duree :  <= 137.500000 
					class : 1 {-1: 94, 1: 109}
			var budget :  <= 36500000.000000 
				var Sci-Fi :  > 0.500000 
					class : -1 {-1: 5, 1: 4}
				var Sci-Fi :  <= 0.500000 
					class : 1 {1: 252, -1: 31}
		var Duree :  <= 115.500000 
			var Biography :  > 0.500000 
				var annee :  > 2011.500000 
					class : -1 {-1: 2}
				var annee :  <= 2011.500000 
					class : 1 {1: 24, -1: 2}
			var Biography :  <= 0.500000 
				var Thriller :  > 0.500000 
					class : -1 {-1: 162, 1: 66}
				var Thriller :  <= 0.500000 
					class : -1 {1: 135, -1: 149}
	var budget :  <= 15660000.000000 
		var annee :  > 1973.500000 
			var Duree :  > 104.500000 
				var budget :  > 11158052.500000 
					class : 1 {1: 131, -1: 39}
				var budget :  <= 11158052.500000 
					class : 1 {-1: 25, 1: 321}
			va